<a href="https://colab.research.google.com/github/SizovV/Music_Genres/blob/main/compute_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Вычисление VGG эмбеддингов для fma датасета.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#Importing and Testing the VGGish System

Based on the directions at: https://github.com/tensorflow/models/tree/master/research/audioset

In [ ]:
!pip install SoundFile

In [ ]:
!pip install librosa

In [ ]:
!pip install audiofile

In [ ]:
!pip install numpy scipy
!pip install resampy tensorflow six

In [ ]:
pip install pydub

In [ ]:
! apt-get install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [ ]:
# Copy the source files to the current directory.
!cp drive/MyDrive/data/vgg/* .

In [ ]:
!pip install tqdm

In [ ]:
pip install tf-slim

In [ ]:
# Run the test, which also loads all the necessary functions.
from vggish_smoke_test import *


Testing your install of VGGish

Log Mel Spectrogram example:  [[-4.47303259 -4.29463765 -4.14939193 ... -3.97474254 -3.94778045
  -3.78685566]
 [-4.48592983 -4.28831745 -4.13994942 ... -3.98374974 -3.94981089
  -3.79512755]
 [-4.46165595 -4.29335712 -4.14907932 ... -3.96438562 -3.9489109
  -3.78621325]
 ...
 [-4.46165595 -4.29335712 -4.14907932 ... -3.96438562 -3.9489109
  -3.78621325]
 [-4.46165595 -4.29335712 -4.14907932 ... -3.96438562 -3.9489109
  -3.78621325]
 [-4.46165595 -4.29335712 -4.14907932 ... -3.96438562 -3.9489109
  -3.78621325]]
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Flatten instead.
INFO:tensorflow:Restoring parameters from vggish_model.ckpt
VGGish embedding:  [-0.4325209  -0.25330514 -0.0389193  -0.16376004 -0.34991807 -0.59936917
 -0.05658104  0.16280249 -0.7555176  -0.08260247 -0.03138605 -0.8314716
 -0.10581706 -0.01420264 -0.11077993 -0.06599088 -0.22666311  0.8060125
 -0.56459844 -0.07349

#Using the VGGish System

In [ ]:
import vggish_slim
import vggish_params
import vggish_input

def CreateVGGishNetwork(hop_size=0.96):   # Hop size is in seconds.
  """Define VGGish model, load the checkpoint, and return a dictionary that points
  to the different tensors defined by the model.
  """
  vggish_slim.define_vggish_slim()
  checkpoint_path = 'vggish_model.ckpt'
  vggish_params.EXAMPLE_HOP_SECONDS = hop_size
  
  vggish_slim.load_vggish_slim_checkpoint(sess, checkpoint_path)

  features_tensor = sess.graph.get_tensor_by_name(
      vggish_params.INPUT_TENSOR_NAME)
  embedding_tensor = sess.graph.get_tensor_by_name(
      vggish_params.OUTPUT_TENSOR_NAME)
    
  return {'features': features_tensor,
          'embedding': embedding_tensor}

In [ ]:
def ProcessWithVGGish(vgg, x):
  '''Run the VGGish model, starting with a path to the sound x.'''

  # Produce a batch of log mel spectrogram examples.
  input_batch = vggish_input.wavfile_to_examples(x)
  # print('Log Mel Spectrogram example: ', input_batch[0])

  [embedding_batch] = sess.run([vgg['embedding']],
                               feed_dict={vgg['features']: input_batch})

  return embedding_batch


In [ ]:
# Test these new functions with the original test.
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
sess = tf.Session()
tf.disable_eager_execution()

vgg = CreateVGGishNetwork(0.960)

Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from vggish_model.ckpt


In [ ]:
def get_audio_path(audio_dir, track_id):
    """
    Return the path to the mp3 given the directory where the audio is stored
    and the track ID.
    Examples
    --------
    >>> import utils
    >>> AUDIO_DIR = os.environ.get('AUDIO_DIR')
    >>> utils.get_audio_path(AUDIO_DIR, 2)
    '../data/fma_small/000/000002.mp3'
    """
    tid_str = '{:06d}'.format(track_id)
    return os.path.join(audio_dir, tid_str[:3], tid_str + '.mp3')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import os

In [ ]:
from pydub import AudioSegment

In [ ]:
data_dir = 'drive/MyDrive/data'
music_dir = os.path.join(data_dir, 'fma_small')
embedding_dir = os.path.join(data_dir, 'embeddings')

In [ ]:
tracks = pd.read_csv(os.path.join(data_dir, 'batch_2.csv'), index_col=0, names=['n', 'id']).iloc[:, 0].to_numpy()

In [ ]:
track_id = 98569
test_track = get_audio_path(music_dir, track_id)
sound = AudioSegment.from_mp3(test_track)
test_track_wav = test_track.split('.')[0] + '.wav'
sound.export(test_track_wav, format="wav")

<_io.BufferedRandom name='drive/MyDrive/data/fma_small/098/098569.wav'>

In [ ]:
import soundfile as sf

In [ ]:
y, sr = sf.read(test_track_wav)

In [ ]:
y.shape

(67439, 2)

In [ ]:
np.count_nonzero(y!=0)

134641

In [ ]:
test_batch = ProcessWithVGGish(vgg, test_track_wav)

In [ ]:
test_batch.shape

(1, 128)

In [ ]:
np.save(os.path.join(embedding_dir, '{}.npy'.format(track_id)), test_batch)

In [ ]:
!pip install tqdm

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
for track_id in tqdm(tracks):
  track = get_audio_path(music_dir, track_id)
  sound = AudioSegment.from_mp3(track)
  track_wav = track.split('.')[0] + '.wav'
  sound.export(track_wav, format="wav")
  batch = ProcessWithVGGish(vgg, track_wav)
  np.save(os.path.join(embedding_dir, '{}.npy'.format(track_id)), batch)